In [60]:
import pandas as pd 

df=pd.read_csv("/Users/amangolani/Downloads/uncleaned_final.csv")


In [61]:
df.columns

Index(['Unnamed: 0', 'Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA',
       'EBIT', 'Net Interest Income', 'Interest Expense', 'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Diluted Average Shares', 'Basic Average Shares',
       'Diluted EPS', 'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Net Income', 'Minority Interests',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
       'Other Non Operating Income Expenses', 'Special Income Charges',
       'Net Non Operating Interest Income Expense',
       'Interest Expense Non Operating', 'Operating Income',
       'Operating Expense', 'Other Operating Expenses',
       'Depr

In [62]:
df=df.rename(columns={'Unnamed: 0':'quarter_end'})

In [30]:
df.columns

Index(['quarter_end', 'Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA',
       'EBIT', 'Net Interest Income', 'Interest Expense', 'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Diluted Average Shares', 'Basic Average Shares',
       'Diluted EPS', 'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Net Income', 'Minority Interests',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
       'Other Non Operating Income Expenses', 'Special Income Charges',
       'Net Non Operating Interest Income Expense',
       'Interest Expense Non Operating', 'Operating Income',
       'Operating Expense', 'Other Operating Expenses',
       'Dep

In [21]:
if (df['Total Revenue'].any()!=df['Operating Revenue'].any()): print("Nope")

In [22]:
df=df.drop(columns=['Total Revenue'])

In [23]:
df['Operating Revenue']

0      5.274407e+11
1      5.570699e+11
2      5.323128e+11
3      5.348973e+11
4      5.441235e+11
           ...     
644    1.030837e+10
645    9.312470e+09
646    9.434360e+09
647    9.565550e+09
648    8.610770e+09
Name: Operating Revenue, Length: 649, dtype: float64

In [ ]:
# 0) Prep

df["quarter_end"] = pd.to_datetime(df["quarter_end"])
df = df.sort_values(["Ticker", "quarter_end"])
g = df.groupby("Ticker")

# 1) Core signals
df["rev_qoq"]        = g["Total Revenue"].pct_change(1)
df["rev_yoy"]        = g["Total Revenue"].pct_change(4)
df["rev_ttm"]        = g["Total Revenue"].rolling(4).sum().reset_index(level=0, drop=True)
df["rev_ttm_change"] = g["rev_ttm"].pct_change(1)

df["gross_margin"]   = df["Gross Profit"] / df["Total Revenue"]
df["ebitda_margin"]  = df["EBITDA"] / df["Total Revenue"]
df["d_gross_margin"] = g["gross_margin"].diff(1)
df["d_ebitda_margin"]= g["ebitda_margin"].diff(1)

# 2) Lags (1,2,3) for features you want
feat_list = ["rev_qoq","rev_yoy","rev_ttm_change","gross_margin","d_gross_margin",
             "ebitda_margin","d_ebitda_margin"]
for f in feat_list:
    for L in (1,2,3):
        df[f"{f}_l{L}"] = g[f].shift(L)

# Optional window aggregates
df["rev_qoq_mean_3"]      = df[["rev_qoq_l1","rev_qoq_l2","rev_qoq_l3"]].mean(axis=1)
df["gross_margin_trend_3"]= df["gross_margin_l1"] - df["gross_margin_l3"]

# 3) Target (example: next-quarter YoY revenue growth)
df["y_rev_yoy_tplus1"] = g["rev_yoy"].shift(-1)

# 4) Modeling view: keep rows with full 3-lag history and a target
lag_cols = [c for c in df.columns if c.endswith(("_l1","_l2","_l3"))]
model_df = df.dropna(subset=lag_cols + ["y_rev_yoy_tplus1"]).copy()

# Now model_df has rows keyed by (Ticker, quarter_end=t) to predict t+1


/var/folders/9s/531sylxs0mz8lk88p22j84t00000gn/T/ipykernel_6803/936401889.py:11: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["rev_ttm_change"] = g["rev_ttm"].pct_change(1)


In [64]:
df.columns

Index(['quarter_end', 'Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA',
       'EBIT', 'Net Interest Income', 'Interest Expense', 'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Diluted Average Shares', 'Basic Average Shares',
       'Diluted EPS', 'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Net Income', 'Minority Interests',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
       'Other Non Operating Income Expenses', 'Special Income Charges',
       'Net Non Operating Interest Income Expense',
       'Interest Expense Non Operating', 'Operating Income',
       'Operating Expense', 'Other Operating Expenses',
       'Dep

In [65]:
ticker_name={"automobile" : [
    "TATAMOTORS.NS",
    "MARUTI.NS",
    "M&M.NS",
    "BAJAJ-AUTO.NS",
    "HEROMOTOCO.NS",
    "EICHERMOT.NS",
    "TVSMOTOR.NS",
    "ASHOKLEY.NS",
    "BHARATFORG.NS",
    "MRF.NS",
    "APOLLOTYRE.NS",
    "BOSCHLTD.NS",
    "EXIDEIND.NS",
    "AMARAJABAT.NS",
    "SONACOMS.NS",
    "MOTHERSON.NS",
    "UNOMINDA.NS"
],

"banking" : [
    "HDFCBANK.NS",
    "ICICIBANK.NS",
    "SBIN.NS",
    "KOTAKBANK.NS",
    "AXISBANK.NS",
    "INDUSINDBK.NS",
    "BANKBARODA.NS",
    "PNB.NS",
    "IDFCFIRSTB.NS",
    "UNIONBANK.NS",
    "CANBK.NS",
    "FEDERALBNK.NS",
    "BANDHANBNK.NS",
    "YESBANK.NS",
    "INDIANB.NS",
    "RBLBANK.NS"
]
,
"power" : [
    "NTPC.NS",
    "POWERGRID.NS",
    "TATAPOWER.NS",
    "ADANIGREEN.NS",
    "ADANIPOWER.NS",
    "ADANIENSOL.NS", # Adani Energy Solutions (formerly Adani Transmission)
    "JSWENERGY.NS",
    "PFC.NS",
    "RECLTD.NS",
    "BHEL.NS",
    "NHPC.NS",
    "SJVN.NS",
    "TORNTPOWER.NS",
    "SUZLON.NS",
    "IEX.NS" # Indian Energy Exchange
]
,
"telecom" :[
    "BHARTIARTL.NS",
    "RELIANCE.NS", # For Jio Platforms
    "IDEA.NS", # Vodafone Idea
    "INDUSTOWER.NS",

    "TATACOMM.NS",
    "TEJASNET.NS",
    "STLTECH.NS",
    "HFCL.NS",
    "ITI.NS",

    "MTNL.NS",
    "RAILTEL.NS"
]
,
"fmcg": [
    "HINDUNILVR.NS",
    "ITC.NS",
    "NESTLEIND.NS",
    "BRITANNIA.NS",
    "TATACONSUM.NS",
    "DABUR.NS",
    "MARICO.NS",
    "GODREJCP.NS",
    "COLPAL.NS",
    "PGHH.NS",
    "MCDOWELL-N.NS", # United Spirits
    "VBL.NS", # Varun Beverages
    "EMAMILTD.NS",
    "RADICO.NS",
    "JYOTHYLAB.NS"
]
,
"IT" : [
    "TCS.NS",
    "INFY.NS",
    "HCLTECH.NS",
    "WIPRO.NS",
    "TECHM.NS",
    "LTIM.NS", # LTIMindtree
    "PERSISTENT.NS",
    "COFORGE.NS",
    "MPHASIS.NS",
    "TATAELXSI.NS",
    "KPITTECH.NS",
    "LTTS.NS", # L&T Technology Services
    "OFSS.NS", # Oracle Financial Services
    "CYIENT.NS",
    "HAPPSTMNDS.NS"
]
,
"Chemical" : [
    "PIDILITIND.NS",
    "UPL.NS",
    "SRF.NS",
    "AARTIIND.NS",
    "DEEPAKNTR.NS",
    "PIIND.NS",
    "TATACHEM.NS",
    "ATUL.NS",
    "NAVINFLUOR.NS",
    "COROMANDEL.NS",
    "VINATIORGA.NS",
    "GUJFLUORO.NS", # Gujarat Fluorochemicals
    "SUMICHEM.NS",
    "CLEAN.NS",
    "SOLARINDS.NS"
]
,
"Metals Mining": [
    "TATASTEEL.NS",
    "JSWSTEEL.NS",
    "HINDALCO.NS",
    "VEDL.NS",
    "COALINDIA.NS",
    "SAIL.NS",
    "JINDALSTEL.NS",
    "HINDZINC.NS",
    "NATIONALUM.NS",
    "NMDC.NS",
    "APLAPOLLO.NS",
    "HINDCOPPER.NS",
    "JSL.NS", # Jindal Stainless
    "RATNAMANI.NS",
    "WELCORP.NS"
]
,
"Pharma" : ["CIPLA.NS", "DRREDDY.NS", "SUNPHARMA.NS", "LUPIN.NS", "NATCOPHARM.NS", "DIVISLAB.NS", "GLENMARK.NS", "ZYDUSLIFE.NS", "PPLPHARMA.NS", "SIGACHI.NS", "MANKIND.NS", "AJANTPHARM.BO", "JBCHEPHARM.NS", "ABBOTINDIA.BO", "GRANULES.NS", "NEULANDLAB.NS", "LAURUSLABS.NS", "MAXHEALTH.NS", "ZYDUSLIFE.NS", "MEDANTA.NS"]}

In [66]:
# 1) Build a ticker → sector mapping dict
ticker_to_sector = {
    ticker: sector
    for sector, tickers in ticker_name.items()
    for ticker in tickers
}

# 2) Add Sector column from Ticker
df["Sector"] = df["Ticker"].map(ticker_to_sector)

# 3) (Optional) mark anything unmapped
df["Sector"] = df["Sector"].fillna("Unknown")


In [67]:
df['Sector']

94     Chemical
93     Chemical
92     Chemical
91     Chemical
90     Chemical
         ...   
641      Pharma
585      Pharma
640      Pharma
584      Pharma
639      Pharma
Name: Sector, Length: 649, dtype: object

In [68]:
df.columns

Index(['quarter_end', 'Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA',
       'EBIT', 'Net Interest Income', 'Interest Expense', 'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Diluted Average Shares', 'Basic Average Shares',
       'Diluted EPS', 'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Net Income', 'Minority Interests',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
       'Other Non Operating Income Expenses', 'Special Income Charges',
       'Net Non Operating Interest Income Expense',
       'Interest Expense Non Operating', 'Operating Income',
       'Operating Expense', 'Other Operating Expenses',
       'Dep

In [50]:
for sector, tickers in ticker_name.items():
    print(sector)

automobile_list
banking_
power_list
telecom
fmcg
IT
Chemical
Metals Mining
Pharma


In [69]:
# PHASE 1: Predict per-company y_rev_yoy_tplus1 using an LSTM over the last 3 quarters

import pandas as pd
import numpy as np

# If you have TensorFlow installed:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler

# 0) Load & basic prep
# PATH = "/Users/amangolani/Downloads/uncleaned_final.csv"  # your uploaded file
# df = pd.read_csv(PATH)

# # parse dates, sort
# df = df.rename(columns={"Unnamed: 0": "quarter_end"})
# df["quarter_end"] = pd.to_datetime(df["quarter_end"])
# df = df.sort_values(["Ticker", "quarter_end"]).reset_index(drop=True)

# 1) Choose compact, high-signal features for sequence steps (keep it small; data is short)
step_feats = [
    "rev_qoq", "rev_ttm_change", "gross_margin", "ebitda_margin",
    "d_gross_margin", "d_ebitda_margin"
]
target_col = "y_rev_yoy_tplus1"

# Ensure numeric
for c in step_feats + [target_col, "Total Revenue"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# 2) Build 3-step sequences per Ticker (use rows t-2, t-1, t → predict target at t+1)
X_list, y_list, keys = [], [], []
for ticker, g in df.groupby("Ticker", sort=False):
    g = g.sort_values("quarter_end")
    for end_idx in range(2, len(g)-1):  # end_idx is t
        window = g.iloc[end_idx-2:end_idx+1]       # t-2..t (3 rows)
        tgt_next = g.iloc[end_idx+1]["y_rev_yoy_tplus1"]  # target at t+1

        if window[step_feats].isna().any().any() or pd.isna(tgt_next):
            continue

        X_list.append(window[step_feats].values)
        y_list.append(float(tgt_next))
        keys.append({
            "Ticker": ticker,
            "Sector": window.iloc[-1]["Sector"] if "Sector" in window else None,
            "t_quarter_end": window.iloc[-1]["quarter_end"],           # t
            "tplus1_quarter_end": g.iloc[end_idx+1]["quarter_end"]     # t+1
        })

X = np.array(X_list)
y = np.array(y_list)
meta = pd.DataFrame(keys)


print("X shape:", X.shape, "y shape:", y.shape)

# 3) Time-based split (use earliest 70–80% for train, last 20–30% for validation)
# We'll split on the tplus1_quarter_end to avoid leakage.
order = np.argsort(meta["tplus1_quarter_end"].values.astype(np.int64))
X, y, meta = X[order], y[order], meta.iloc[order].reset_index(drop=True)

split_at = int(len(X) * 0.8)
X_train, X_val = X[:split_at], X[split_at:]
y_train, y_val = y[:split_at], y[split_at:]
meta_train, meta_val = meta.iloc[:split_at], meta.iloc[split_at:]

# 4) Scale features per time-step feature dimension (fit scaler on TRAIN ONLY)
# Flatten time dimension to fit a single scaler across features, then reshape back.
n_steps, n_feats = X.shape[1], X.shape[2]
scaler = StandardScaler().fit(X_train.reshape(-1, n_feats))
X_train_scaled = scaler.transform(X_train.reshape(-1, n_feats)).reshape(-1, n_steps, n_feats)
X_val_scaled   = scaler.transform(X_val.reshape(-1, n_feats)).reshape(-1, n_steps, n_feats)

# 5) Define a tiny LSTM (keep it small to avoid overfitting)
model = Sequential([
    LSTM(32, input_shape=(n_steps, n_feats)),
    Dropout(0.2),
    Dense(16, activation="relu"),
    Dense(1)  # predict y_rev_yoy_tplus1 (a real number, e.g., 0.12 for +12%)
])
model.compile(optimizer="adam", loss="mae", metrics=["mse"])  # MAE is robust for growth rates

# 6) Train with early stopping
es = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
hist = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=300, batch_size=32, callbacks=[es], verbose=1
)

# 7) Evaluate
val_preds = model.predict(X_val_scaled).flatten()
val_mae = np.mean(np.abs(val_preds - y_val))
val_mape = np.mean(np.abs((val_preds - y_val) / (np.abs(y_val) + 1e-6)))
print(f"Validation MAE:  {val_mae:.4f}")
print(f"Validation MAPE: {val_mape:.4f}")

# 8) Keep artifacts for Phase 2 (sector roll-up)
# meta_val has t (feature cutoff) and t+1 (prediction quarter) stamps
out = meta_val.copy()
out["yhat_rev_yoy_tplus1"] = val_preds
out["y_true_rev_yoy_tplus1"] = y_val
# save for later rollups or inspection
out.to_csv("/Users/amangolani/Downloads/per_company_yoy_preds.csv", index=False)
print("Saved per-company predictions to /mnt/data/per_company_yoy_preds.csv")


X shape: (2, 3, 6) y shape: (2,)
Epoch 1/300


/Users/amangolani/.pyenv/versions/finance/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.2873 - mse: 0.0825 - val_loss: 0.3341 - val_mse: 0.1116
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.2757 - mse: 0.0760 - val_loss: 0.3259 - val_mse: 0.1062
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.2654 - mse: 0.0704 - val_loss: 0.3177 - val_mse: 0.1010
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.2583 - mse: 0.0667 - val_loss: 0.3102 - val_mse: 0.0962
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.2459 - mse: 0.0605 - val_loss: 0.3028 - val_mse: 0.0917
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.2539 - mse: 0.0645 - val_loss: 0.2943 - val_mse: 0.0866
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.2305 - mse: 0.0531 - val_loss: 0.2859 - val_mse: 0.0817
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.2352 - mse: 0.0553 - val_loss: 0.2768 - val_mse: 0.0766
Epoch 9/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.2305 - mse: 0.0531 - val_l

In [58]:
df.columns

Index(['Unnamed: 0', 'Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA',
       'EBIT', 'Net Interest Income', 'Interest Expense', 'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Diluted Average Shares', 'Basic Average Shares',
       'Diluted EPS', 'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Net Income', 'Minority Interests',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
       'Other Non Operating Income Expenses', 'Special Income Charges',
       'Net Non Operating Interest Income Expense',
       'Interest Expense Non Operating', 'Operating Income',
       'Operating Expense', 'Other Operating Expenses',
       'Depr

In [70]:
print("Total samples built:", len(meta))
print("Train / Val:", len(X_train), len(X_val))
print(meta.groupby("Ticker").size().sort_values(ascending=False).head(20))


Total samples built: 2
Train / Val: 1 1
Ticker
ZYDUSLIFE.NS    2
dtype: int64


In [76]:
df["d_ebitda_margin"]

94          NaN
93    -0.042710
92     0.002954
91    -0.002841
90     0.003535
         ...   
641    0.000000
585    0.022245
640    0.000000
584    0.042601
639    0.000000
Name: d_ebitda_margin, Length: 649, dtype: float64

In [71]:
meta

,Ticker,Sector,t_quarter_end,tplus1_quarter_end
0,ZYDUSLIFE.NS,Pharma,2025-03-31,2025-03-31
1,ZYDUSLIFE.NS,Pharma,2025-03-31,2025-06-30


In [72]:
df

,quarter_end,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,EBITDA,EBIT,Net Interest Income,...,ebitda_margin_l1,ebitda_margin_l2,ebitda_margin_l3,d_ebitda_margin_l1,d_ebitda_margin_l2,d_ebitda_margin_l3,rev_qoq_mean_3,gross_margin_trend_3,y_rev_yoy_tplus1,Sector
94,2024-06-30,0.000000e+00,0.055172,3.110000e+09,1.370000e+09,1.020000e+09,1.157000e+10,3.110000e+09,2.090000e+09,-6.400000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chemical
93,2024-09-30,6.000000e+06,0.300000,2.020000e+09,5.200000e+08,1.080000e+09,1.017000e+10,2.040000e+09,9.600000e+08,-6.200000e+08,...,0.168017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chemical
92,2024-12-31,0.000000e+00,0.300000,2.360000e+09,4.600000e+08,1.110000e+09,1.230000e+10,2.360000e+09,1.250000e+09,-8.500000e+08,...,0.125307,0.168017,NaN,-0.042710,NaN,NaN,-0.120475,NaN,NaN,Chemical
91,2025-03-31,-3.198000e+07,0.300000,2.374600e+09,9.587000e+08,1.132700e+09,1.973350e+10,2.268000e+09,1.135300e+09,-2.626000e+08,...,0.128261,0.125307,0.168017,0.002954,-0.042710,NaN,0.004873,-0.043411,-0.095084,Chemical
90,2025-06-30,0.000000e+00,0.300000,2.160000e+09,4.300000e+08,1.140000e+09,1.122000e+10,2.160000e+09,1.020000e+09,-6.000000e+08,...,0.125420,0.128261,0.125307,-0.002841,0.002954,-0.042710,-0.002491,-0.466569,NaN,Chemical
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,2024-12-31,0.000000e+00,0.151592,1.445100e+10,1.023800e+10,2.290000e+09,1.583400e+10,1.445100e+10,1.216100e+10,-3.200000e+08,...,0.282053,0.299124,0.299124,-0.017071,0.000000,-0.055991,-0.050784,-0.021304,0.230092,Pharma
585,2025-03-31,-5.360871e+08,0.253110,1.990900e+10,1.170900e+10,2.379000e+09,2.025500e+10,1.914100e+10,1.676200e+10,1.752000e+09,...,0.282053,0.282053,0.299124,0.000000,-0.017071,0.000000,0.000645,-0.021304,0.230092,Pharma
640,2025-03-31,-5.360871e+08,0.253110,1.990900e+10,1.170900e+10,2.379000e+09,2.025500e+10,1.914100e+10,1.676200e+10,1.752000e+09,...,0.304299,0.282053,0.282053,0.022245,0.000000,-0.017071,0.076550,-0.012962,0.262223,Pharma
584,2025-06-30,0.000000e+00,0.225971,2.243400e+10,1.466800e+10,2.381000e+09,1.789500e+10,2.243400e+10,2.005300e+10,-8.470000e+08,...,0.304299,0.304299,0.282053,0.000000,0.022245,0.000000,0.075905,-0.012962,0.262223,Pharma


In [74]:
len(set(df["Ticker"]))

129

In [75]:
for feat in step_feats:
    print(df[feat].isna().sum())

129
516
85
80
197
193


In [79]:
df["Sector"].isna().any()

np.False_

In [ ]:
#y_rev_yoy_tplus1
#